In [6]:
from src.datasetSetup import srediDataset
srediDataset("datasetovi", "dataset1.zip")

Skidanje dataseta, unzipovanje, organizovanje (iznad) 

Loading data i ucitavanje pretreniranog modela, detektovanje objekata na slici PRE fine-tuninga

In [2]:
import matplotlib.pyplot as plt
import torchvision
import cv2
from src.customDataset import CustomVOCDataset
from src.modelFunkcije import *
from src.utils import *

In [ ]:
# Definisemo transformations
transform = T.Compose([T.ToTensor()])

# Kreiramo dataset instancu i dataloader
dataset = CustomVOCDataset(root='datasetovi/dataset1.zip_sredjen/valid', CLASSES=classes, transforms=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

# Ucitamo pretrenirani Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Pokrenemo detektovanje nad datasetom koji smo load-ovali
iteration_count = 0
for images, targets in data_loader:

    if iteration_count >= N:
        break

    images = list(img.to(device) for img in images)

    with torch.no_grad():
        prediction = model(images)

    for img, pred in zip(images, prediction):
        plot_image_with_boxes(img, pred['boxes'], pred['labels'], pred['scores'], classes)

    iteration_count += 1

Loading data i fine-tuning modela

In [3]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from src.customDataset import CustomVOCDataset
from src.modelFunkcije import *
from src.utils import *

In [ ]:
# Definisemo transformations
transform = T.Compose([T.ToTensor()])

def my_collate_fn(x): return tuple(zip(*x))

# Kreiramo dataset instancu i dataloader
dataset = CustomVOCDataset(root='datasetovi/dataset1.zip_sredjen/train', CLASSES=CLASSES, transforms=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, collate_fn=my_collate_fn)

# Glavni deo koda za trening
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes)
model.to(device)

train_model(model, data_loader, device, num_epochs)

save_directory = 'finetune modeli'
os.makedirs(save_directory, exist_ok=True)

# Cuva model
torch.save(model.state_dict(), os.path.join(save_directory, 'frcnn_custom_final.pth'))

Plotovanje rezultata

In [4]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
from src.utils import *

In [ ]:
# Kreiramo dataset instancu i dataloader
dataset2 = CustomVOCDataset(root='datasetovi\dataset1.zip_sredjen\test', CLASSES=CLASSES, transforms=transform)
data_loader2 = DataLoader(dataset2, batch_size=1, shuffle=True, num_workers=4, collate_fn=my_collate_fn)

model.eval()

# Pokrenemo detektovanje nad datasetom koji smo load-ovali
iteration_count = 0
for images, targets in data_loader2:

    if iteration_count >= N:
        break

    images = list(img.to(device) for img in images)
    with torch.no_grad():
        prediction = model(images)

    for img, pred in zip(images, prediction):
        plot_image_with_boxes(img, pred['boxes'], pred['labels'], pred['scores'], CLASSES)

    iteration_count += 1